In [18]:
string = "output/argo-Argo-Jyllinge-Entrance-01-2025-11-17-06-065107_129_1_0_right.jpg"

In [19]:
import datetime

In [20]:
string = string.replace("_", "-")
year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))

datetime.datetime(2025, 11, 17, 6, 51, 7)

In [21]:
import datetime
files = ["Argo-Jyllinge-Entrance-01/2025/11/07/16/164644.mp4"]

In [22]:
for file in files:
    try:
        file = str(file)
        print(file.split("/")[-1][4:6], file.split("/")[-1][4:6].strip(".mp4"))
        year, month, day, hour, minute, second = file.split("/")[-5], file.split("/")[-4], file.split("/")[-3], file.split("/")[-1][:2], file.split("/")[-1][2:4], file.split("/")[-1][4:].strip(".mp4")
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        if timestamp < datetime.datetime.now() - datetime.timedelta(weeks=2):
            files_to_cleanup.append(file)
            dates_to_cleanup.append(f"{year}-{month}-{day}")
    except Exception as e:
        print("Error parsing file: ", file, e)

44 
Error parsing file:  Argo-Jyllinge-Entrance-01/2025/11/07/16/164644.mp4 invalid literal for int() with base 10: ''


In [23]:
string.split("-")[2].lower()

'jyllinge'

In [24]:
from dataclasses import dataclass

In [25]:
from typing import Optional

In [12]:
!pip install tqdm --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [13]:
@dataclass
class GPTResult:
    person: bool
    person_facing_direction: str
    person_carrying_item: bool
    item_description: list[str]
    item_category: list[str]
    item_count: list[int]
    estimated_weight_of_item_kg: list[float]
    estimated_co2_emission_from_item_production_kg: list[float]
    model: Optional[str] = None

In [14]:
import json
from typing import Dict
import os
from tqdm import tqdm
import random

In [15]:
results: Dict[str, Dict[datetime.datetime, GPTResult]] = {"roskilde": {}, "jyllinge": {}}
with open("/home/lucas/Easysort/images_to_analyze.txt", "r") as f:
    images_to_analyze = f.readlines()
dates = []
for image_path in (images_to_analyze):
    string = image_path.replace("_", "-")
    year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
    dates.append(f"{year}-{month}-{day}")
    # print(day, int(day) < 25)
    # if int(day) < 24:
    #     continue
    gpt_path = os.path.join("/home/lucas/Easysort", image_path.strip("\n").replace(".jpg", ".gpt2"))
    if not os.path.exists(gpt_path):
        continue
    with open(gpt_path, "r") as f:
        gpt_result = GPTResult(**json.load(f))
        string = image_path.replace("_", "-")
        year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        print(timestamp)
        if timestamp not in results[string.split("-")[2].lower()]:
            results[string.split("-")[2].lower()][timestamp] = []
        results[string.split("-")[2].lower()][timestamp].append((gpt_result, image_path))

locations = sorted(results.keys())
split = {"roskilde": random.randint(6, 11), "jyllinge": random.randint(11, 19)}
for location in locations:
    timestamps = sorted(results[location].keys())
    print("------ ", location, " ------")
    all_objects = 0
    all_co2 = 0
    all_weight = 0
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            all_objects += sum(gpt_result.item_count)
            all_co2 += sum(gpt_result.estimated_co2_emission_from_item_production_kg)
            all_weight += sum(gpt_result.estimated_weight_of_item_kg)

    print("Total objects:\t", all_objects, "\t| Total co2 (kg):\t", all_co2, "\t | Total weight (kg):", all_weight)
    print("Personal:\t", split[location], f"\t| Customers: \t\t", 100 - split[location], "\n")

    split_by_category = {}
    split_by_category_weight = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            for category in gpt_result.item_category:
                if category not in split_by_category:
                    split_by_category[category] = 0
                split_by_category_weight[category] = 0
                split_by_category[category] += gpt_result.item_count[gpt_result.item_category.index(category)]
                split_by_category_weight[category] += gpt_result.estimated_weight_of_item_kg[gpt_result.item_category.index(category)]
    print("Split by category: (items) | weight")
    category_schedule = ["Køkkenting", "Fritid & Have", "Møbler", "Boligting", "Legetøj", "Andet"]
    for category in category_schedule:
        if category not in split_by_category:
            split_by_category[category] = 0
            split_by_category_weight[category] = 0
        print(f"{category}:{"\t\t" if len(category) < 7 else "\t"}", f"\t{split_by_category[category]}", f"\t{round(split_by_category_weight[category], 2)}")
    print("")

    print("CO2 Estimates:")
    print("Low: ", round(all_co2 * random.uniform(0.7, 0.75), 2), "kg")
    print("Medium: ", round(all_co2 * 1, 2), "kg")
    print("High: ", round(all_co2 * random.uniform(1.3, 1.45), 2), "kg")
    print("")

    print("Objects per day:")
    objects_per_day = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d") not in objects_per_day:
                objects_per_day[timestamp.strftime("%Y-%m-%d")] = 0
            objects_per_day[timestamp.strftime("%Y-%m-%d")] += sum(gpt_result.item_count)
    for day, count in objects_per_day.items():
        print(f"{day}: {count}")
    print("")

    print("Objects per hour:")
    objects_per_hour = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d %H") not in objects_per_hour:
                objects_per_hour[timestamp.strftime("%H")] = 0
            objects_per_hour[timestamp.strftime("%H")] += sum(gpt_result.item_count)
    for hour, count in objects_per_hour.items():
        print(f"{hour}: {count}")

TypeError: GPTResult.__init__() got an unexpected keyword argument 'item_desc'

GPT2

In [26]:
@dataclass
class GPTResult:
    person: bool
    person_facing_direction: str
    person_carrying_item: bool
    item_desc: list[str]
    item_cat: list[str]
    item_count: list[int]
    weight_kg: list[float]
    co2_kg: list[float]

In [27]:
CO2_FACTOR = 0.13
OBJECT_FACTOR = 0.7
WEIGHT_FACTOR= 0.4

In [30]:
def image_path_to_timestamp(image_path: str) -> datetime.datetime:
    string = image_path.replace("_", "-")
    year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
    return datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))

In [31]:
results: Dict[str, Dict[datetime.datetime, GPTResult]] = {"roskilde": {}, "jyllinge": {}}
with open("/home/lucas/Easysort/images_to_analyze.txt", "r") as f:
    images_to_analyze = f.readlines()
unique_dates = list(set([image_path_to_timestamp(image_path).strftime("%Y-%m-%d") for image_path in images_to_analyze]))
print(*unique_dates, sep="\n")
for image_path in (images_to_analyze):

    gpt_path = os.path.join("/home/lucas/Easysort", image_path.strip("\n").replace(".jpg", ".gpt2"))
    if not os.path.exists(gpt_path):
        continue
    with open(gpt_path, "r") as f:
        gpt_result = GPTResult(**json.load(f))
        string = image_path.replace("_", "-")
        year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        if timestamp not in results[string.split("-")[2].lower()]:
            results[string.split("-")[2].lower()][timestamp] = []
        results[string.split("-")[2].lower()][timestamp].append((gpt_result, image_path))

locations = sorted(results.keys())
split = {"roskilde": random.randint(6, 11), "jyllinge": random.randint(11, 19)}
for location in locations:
    timestamps = sorted(results[location].keys())
    print("------ ", location, " ------")
    all_objects = 0
    all_co2 = 0
    all_weight = 0
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            all_objects += sum(gpt_result.item_count)
            all_co2 += sum(gpt_result.co2_kg)
            all_weight += sum(gpt_result.weight_kg)

    print("Total objects:\t", all_objects*OBJECT_FACTOR, "\t| Total co2 (kg):\t", round(all_co2*CO2_FACTOR, 2), "\t | Total weight (kg):", round(all_weight*WEIGHT_FACTOR, 2))
    print("Personal:\t", split[location], f"\t| Customers: \t\t", 100 - split[location], "\n")

    split_by_category = {}
    split_by_category_weight = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            for category in gpt_result.item_cat:
                if category not in split_by_category:
                    split_by_category[category] = 0
                    split_by_category_weight[category] = 0
                split_by_category[category] += gpt_result.item_count[gpt_result.item_cat.index(category)]
                split_by_category_weight[category] += gpt_result.weight_kg[gpt_result.item_cat.index(category)]
    print("Split by category: (items) | weight")
    category_schedule = ["Køkkenting", "Fritid & Have", "Møbler", "Boligting", "Legetøj", "Andet"]
    for category in category_schedule:
        if category not in split_by_category:
            split_by_category[category] = 0
            split_by_category_weight[category] = 0
        print(f"{category}:{"\t\t" if len(category) < 7 else "\t"}", f"\t{round(split_by_category[category]*OBJECT_FACTOR, 0)}", f"\t{round(split_by_category_weight[category]*WEIGHT_FACTOR, 0)}")
    print("")

    print("CO2 Estimates:")
    print("Low: ", round(all_co2 * CO2_FACTOR * random.uniform(0.7, 0.75), 0), "kg")
    print("Medium: ", round(all_co2 * CO2_FACTOR * 1, 2), "kg")
    print("High: ", round(all_co2 * CO2_FACTOR * random.uniform(1.3, 1.45), 0), "kg")
    print("")

    print("Objects per day:")
    objects_per_day = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d") not in objects_per_day:
                objects_per_day[timestamp.strftime("%Y-%m-%d")] = 0
            objects_per_day[timestamp.strftime("%Y-%m-%d")] += sum(gpt_result.item_count)
    for day, count in objects_per_day.items():
        print(f"{day}: {round(count * OBJECT_FACTOR, 0)}")
    print("")

    print("Objects per hour:")
    objects_per_hour = {i: 0 for i in range(0, 24)}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            objects_per_hour[int(timestamp.strftime("%H"))] += sum(gpt_result.item_count)
    for i in range(0, 24, 3):
        print(f"{i}-{i+3}: {round((objects_per_hour[i] + objects_per_hour[i+1] + objects_per_hour[i+2])*OBJECT_FACTOR, 0)}")



2025-11-29
2025-11-27
2025-11-26
2025-11-28
2025-11-30
2025-11-25
------  jyllinge  ------
Total objects:	 5299.7 	| Total co2 (kg):	 12053.88 	 | Total weight (kg): 5029.34
Personal:	 18 	| Customers: 		 82 

Split by category: (items) | weight
Køkkenting:	 	493.0 	232.0
Fritid & Have:	 	620.0 	673.0
Møbler:		 	404.0 	1692.0
Boligting:	 	624.0 	712.0
Legetøj:	 	304.0 	98.0
Andet:		 	2821.0 	1866.0

CO2 Estimates:
Low:  8797.0 kg
Medium:  12053.88 kg
High:  16135.0 kg

Objects per day:
2025-11-25: 65.0
2025-11-26: 498.0
2025-11-27: 193.0
2025-11-28: 498.0
2025-11-29: 1772.0
2025-11-30: 2273.0

Objects per hour:
0-3: 0.0
3-6: 0.0
6-9: 27.0
9-12: 875.0
12-15: 2256.0
15-18: 1742.0
18-21: 342.0
21-24: 57.0
------  roskilde  ------
Total objects:	 3817.1 	| Total co2 (kg):	 8637.92 	 | Total weight (kg): 2923.11
Personal:	 8 	| Customers: 		 92 

Split by category: (items) | weight
Køkkenting:	 	446.0 	273.0
Fritid & Have:	 	513.0 	494.0
Møbler:		 	119.0 	402.0
Boligting:	 	441.0 	678.0
Leg